In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim


def load_and_preprocess_image(image_path):
    # Define the same transforms you used during training
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

In [12]:
def predict_image(model, image_tensor):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
    return predicted.item(), probabilities[0]


class ChestXRayClassifier(nn.Module):
    def __init__(self):
        super(ChestXRayClassifier, self).__init__()
        self.resnet = resnet18(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)

# Load the saved model
model = ChestXRayClassifier()
if torch.cuda.is_available():
    model.load_state_dict(torch.load('/content/drive/MyDrive/chest_xray_classifier.pth'))
else:
    model.load_state_dict(torch.load('/content/drive/MyDrive/chest_xray_classifier.pth', map_location=torch.device('cpu')))
model.to('cuda' if torch.cuda.is_available() else 'cpu')

ChestXRayClassifier(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [23]:
def classify_image(image_path):
    # Load and preprocess the image
    image_tensor = load_and_preprocess_image(image_path)

    # Move the image tensor to the same device as the model
    device = next(model.parameters()).device
    image_tensor = image_tensor.to(device)

    # Make prediction
    prediction, probabilities = predict_image(model, image_tensor)

    # Interpret results
    class_names = ['Chest X-ray','Non-Chest X-ray', ]
    predicted_class = class_names[prediction]
    confidence = probabilities[prediction].item()

    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}")
    print(f"Probabilities: X-ray: {probabilities[0]:.2f}, non Chest X-ray: {probabilities[1]:.2f}")



In [24]:
test_images = [
    '/content/drive/MyDrive/images/apple.png',
    '/content/drive/MyDrive/images/chest.png',
    '/content/drive/MyDrive/images/chest2.png',
    '/content/drive/MyDrive/images/chest3.png',
    '/content/drive/MyDrive/images/hand.jpg',
    '/content/drive/MyDrive/images/spookySkeleton .png',

]

for image_path in test_images:
    print(f"\nClassifying image: {image_path}")
    classify_image(image_path)


Classifying image: /content/drive/MyDrive/images/apple.png
Predicted class: Non-Chest X-ray
Confidence: 0.70
Probabilities: X-ray: 0.30, non Chest X-ray: 0.70

Classifying image: /content/drive/MyDrive/images/chest.png
Predicted class: Chest X-ray
Confidence: 1.00
Probabilities: X-ray: 1.00, non Chest X-ray: 0.00

Classifying image: /content/drive/MyDrive/images/chest2.png
Predicted class: Chest X-ray
Confidence: 1.00
Probabilities: X-ray: 1.00, non Chest X-ray: 0.00

Classifying image: /content/drive/MyDrive/images/chest3.png
Predicted class: Chest X-ray
Confidence: 1.00
Probabilities: X-ray: 1.00, non Chest X-ray: 0.00

Classifying image: /content/drive/MyDrive/images/hand.jpg
Predicted class: Non-Chest X-ray
Confidence: 1.00
Probabilities: X-ray: 0.00, non Chest X-ray: 1.00

Classifying image: /content/drive/MyDrive/images/spookySkeleton .png
Predicted class: Non-Chest X-ray
Confidence: 1.00
Probabilities: X-ray: 0.00, non Chest X-ray: 1.00
